## This notebook is an example of how to pipeline two models. 
This notebook is an example how to use DeGirum PySDK to do AI inference of a graphical file using 
two AI models: license plate detection and license recognition. The license plate detection model 
is run on the image and the results are then processed by the license recognition model, 
one license plate at a time. Combined result is then displayed.
This script uses PIL as image processing backend.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you just need to uncomment **one** of the lines in the code below.

You also need to specify your cloud API access token, cloud zoo URLs, and AI server hostname in [env.ini](env.ini) file, located in the same directory as this notebook.

#### Specify where do you want to run your inferences

In [ ]:
import degirum as dg, dgtools

#
# Please UNCOMMENT only ONE of the following lines to specify where to run AI inference
#

target = dg.CLOUD # <-- on the Cloud Platform
# target = dgtools.get_ai_server_hostname() # <-- on AI Server deployed in your LAN
# target = dg.LOCAL # <-- on ORCA accelerator installed on this computer

# connect to AI inference engine getting zoo URL and token from env.ini file
zoo = dg.connect(target, dgtools.get_cloud_zoo_url(), dgtools.get_token())

#### The rest of the cells below should run without any modifications

In [ ]:
# load models for DeGirum Orca AI accelerator
with zoo.load_model("yolo_v5s_lp_det--512x512_quant_n2x_orca1_1") as lp_det_model, \
     zoo.load_model("yolo_v5s_lp_ocr--256x256_quant_n2x_orca1_1") as lp_ocr_model:

     # adjust some model properties
     lp_det_model.overlay_font_scale = 1.5
     lp_det_model.overlay_alpha = 1
     lp_det_model.image_backend = lp_ocr_model.image_backend = "pil"

     # detect license plate boxes
     license_plates = lp_det_model('../../images/Car.jpg')

     # for each detected license plate box
     for license_plate_box in license_plates.results:
          # recognize license plate number inside the box
          license_number = lp_ocr_model(license_plates.image.crop(license_plate_box['bbox']))
          # change object class label to recognized number for nice display
          license_plate_box["label"] = license_number.results[0]["label"]

     # display combined results
     display(license_plates.image_overlay)